In [1]:
!pip install huggingface_hub transformers

In [7]:
from transformers import TrOCRProcessor, ViTImageProcessor, RobertaTokenizer, TrOCRForCausalLM
from huggingface_hub import HfApi, HfFolder, Repository, create_repo, login

In [8]:
model_dir = "/TrOCR-Pipeline/TrOCR-Tibetan/checkpoint-48000"
hf_repo_name = "ta4tsering/Lhasa_Kanjur_TrOCR_model"

In [9]:
token = ""# Replace this with your actual Hugging Face token
login(token=token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [11]:
def upload_to_hf(model_dir, hf_repo_name):
    api = HfApi()
    encode, decode = "google/vit-base-patch16-224-in21k", "sangjeedondrub/tibetan-roberta-base"
    feature_extractor = ViTImageProcessor.from_pretrained(encode)
    tokenizer = RobertaTokenizer.from_pretrained(decode)
    processor = TrOCRProcessor(image_processor=feature_extractor, tokenizer=tokenizer)
    
    # Save the processor locally
    processor.save_pretrained(model_dir)
    
    # Load the trained model
    model = TrOCRForCausalLM.from_pretrained(model_dir)
    
    # Save the model locally
    model.save_pretrained(model_dir)
    # Create the repository if it doesn't exist
    try:
        api.create_repo(repo_id=hf_repo_name, repo_type="model")
    except Exception as e:
        print(f"Repository already exists or another error occurred: {e}")

    # Upload the model
    api.upload_folder(
        folder_path=model_dir,
        path_in_repo=".",
        repo_id=hf_repo_name,
        repo_type="model"
    )
    print(f"Model uploaded to: https://huggingface.co/{hf_repo_name}")

# Upload the best checkpoint
upload_to_hf(model_dir, hf_repo_name)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 170, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


Repository already exists or another error occurred: 409 Client Error: Conflict for url: https://huggingface.co/api/repos/create (Request ID: Root=1-6644bef1-3d8f955101f455c45c76fe15;485de017-0e15-4ee9-964a-69ca84032214)

You already created this model repo


optimizer.pt:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

Model uploaded to: https://huggingface.co/ta4tsering/Lhasa_Kanjur_TrOCR_model
